In [10]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob

In [2]:
lr = 0.001
batch_size = 10
epochs = 10

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

print(device)

cpu


Task 1-2

In [12]:
train_path='C:/Repository/lab/train'
test_path='C:/Repository/lab/test'
val_path='C:/Repository/lab/val'


df = pd.read_csv('paths.csv', sep = ',', header=None)


df.drop(1, axis=1, inplace=True)
df.rename(columns={0: 'absolute_way', 2: 'class_img'}, inplace=True)
df.reset_index(inplace=True)
print(df)


if not os.path.isdir(train_path):
    os.mkdir(train_path)

if not os.path.isdir(test_path):
    os.mkdir(test_path)

if not os.path.isdir(val_path):
    os.mkdir(val_path)

def load_train(df: pd.core.frame.DataFrame, path: str, i: int) -> None:    
    '''
    Загружает i-ую картинку из датафрейма по заданному пути path
    Parameters
        df(pd.core.frame.DataFrame): 
         датафрейм с картинками 
        path(str): 
         путь загрузки картинок
        i(int): 
         номер картинки 
    '''
    image_path = df.absolute_way[i]
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(path, f'{df.class_img[i]}.{i}.jpg'), image)

def load_val(df: pd.core.frame.DataFrame, path: str, i: int) -> None:
    '''
    Загружает i-ую картинку из датафрейма по заданному пути path
    Parameters
        df(pd.core.frame.DataFrame): 
         датафрейм с картинками 
        path(str): 
         путь загрузки картинок
        i(int): 
         номер картинки 
    '''
    image_path = df.absolute_way[i]
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)

def load_test(df: pd.core.frame.DataFrame, path: str, i: int) -> None:
    '''
    Загружает i-ую картинку из датафрейма по заданному пути path
    Parameters
        df(pd.core.frame.DataFrame): 
         датафрейм с картинками 
        path(str): 
         путь загрузки картинок
        i(int): 
         номер картинки 
    '''
    image_path = df.absolute_way[i]
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)


for i in range(800):
    load_train(df, train_path, i)
for i in range(1000,1800):
    load_train(df, train_path, i)
for i in range(800,900):
    load_test(df,test_path,i)
for i in range(1800,1900):
    load_test(df,test_path,i)
for i in range(900,1000):
    load_val(df,val_path,i)
for i in range(1900,1999):
    load_val(df,val_path,i)

train_list = glob.glob(os.path.join(train_path,'*.jpg'))
test_list = glob.glob(os.path.join(test_path, '*.jpg'))

train_list, val_list = train_test_split(train_list, test_size=0.1)

      index                            absolute_way class_img
0         0  C:\Repository\lab\dataset\cat\0000.jpg       cat
1         1  C:\Repository\lab\dataset\cat\0001.jpg       cat
2         2  C:\Repository\lab\dataset\cat\0002.jpg       cat
3         3  C:\Repository\lab\dataset\cat\0003.jpg       cat
4         4  C:\Repository\lab\dataset\cat\0004.jpg       cat
...     ...                                     ...       ...
1995   1995  C:\Repository\lab\dataset\dog\0995.jpg       dog
1996   1996  C:\Repository\lab\dataset\dog\0996.jpg       dog
1997   1997  C:\Repository\lab\dataset\dog\0997.jpg       dog
1998   1998  C:\Repository\lab\dataset\dog\0998.jpg       dog
1999   1999  C:\Repository\lab\dataset\dog\0999.jpg       dog

[2000 rows x 3 columns]
